In [1]:
# %pip install --upgrade --quiet  langchain-pinecone langchain-openai langchain
# %pip install --upgrade --quiet  langchain sentence_transformers
# %pip install --upgrade --quiet  gpt4all > /dev/null

In [1]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/usr/local/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [2]:
f = open("./knowledge_base.txt",'r')
content = f.read()

chunks = content.split("$$")
chunks 

res = embeddings.embed_documents(chunks)
len(res), len(res[0])

(22, 384)

In [3]:
# from langchain_pinecone import Pinecone
from pinecone import Pinecone
from pinecone import PodSpec

api_key = "2b6a0552-7fae-43bb-af69-eb909be154f5"
environment = "gcp-starter"

pc = Pinecone(api_key=api_key)
spec = PodSpec(environment=environment)

index_name = "pharma-llama-kb"

# if index_name in pc.list_indexes().names():
#     pc.delete_index(index_name)

if index_name not in pc.list_indexes().names():
    # we create a new index
    pc.create_index(
            index_name,
            dimension=384,  # dimensionality of miniLM-L6
            metric='dotproduct',
            spec=spec
        )

In [7]:
index = pc.Index(index_name)

# view index stats
index.describe_index_stats()


{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [4]:
len(chunks)

22

In [8]:
from tqdm.auto import tqdm
from uuid import uuid4

batch_limit = 100

texts = []
idx = []
ids = []
metadata = []

for i, text in enumerate(chunks):
    texts.extend(text)
    idx.extend(str(i))
    metadata.extend([{'text': text}])
    
    # if we have reached the batch_limit we can add texts
    if len(texts) >= batch_limit:
        ids = [str(uuid4()) for _ in range(len(texts))]
        embeds = embeddings.embed_documents(texts)
        index.upsert(vectors=zip(ids, embeds, metadata))
        texts = []
        idx = []
        ids = []
        metadata = []
        
if len(texts) > 0 :
    ids = [str(uuid4()) for _ in range(len(texts))]
    embeds = embeddings.embed_documents(texts)
    index.upsert(vectors=zip(ids, embeds, metadata))


In [22]:
from langchain.vectorstores import Pinecone

text_field = "text"  # the metadata field that contains our text

# initialize the vector store object
vectorstore = Pinecone(
    index, embeddings.embed_query, text_field
)

In [24]:
query = ""

vectorstore.similarity_search(
    query,  # our search query
    k=10 
)

[Document(page_content='\nCK-MB (ng/mL):\n- Good: <3 males; <2 females\n- Moderate: 3-4 males; 2-3 females  \n- Poor: 5-10 males; 3.5-6 females\n- Dangerous: >10 males; >6 females\n'),
 Document(page_content='Blood Pressure:\nNormal: Less than 120/80 mmHg\nElevated: 120-129/80-89 mmHg\nHigh Blood Pressure Stage 1: 130-139/80-89 mmHg\nHigh Blood Pressure Stage 2: 140/90 mmHg or higher\nHypotension: Less than 90/60 mmHg\n'),
 Document(page_content='\nCholesterol Levels:\nDesirable: Under 200 mg/dL\nBorderline High: 200-239 mg/dL\nHigh: 240 mg/dL and above\n'),
 Document(page_content='\nECG:\nNormal: Normal rhythm, rate 60-100 bpm, normal waveform\nAbnormal: Arrhythmias, slowed/fast heart rate, ST segment changes, T wave inversion\n'),
 Document(page_content='\nTroponin (ng/mL):\n- Good: <0.01  \n- Moderate: 0.01-0.03\n- Poor: 0.04-0.06 \n- Dangerous: >0.06\n'),
 Document(page_content='\nCreatinine (mg/dL):\n- Good: 0.6-1.0 \n- Moderate: 1.1-1.3\n- Poor: 1.4-1.9\n- Dangerous: ≥2.0\n'),
 D